# Lab 10. NN, ReLu, Xavier, Dropout and Adam
---

### Load MNIST data

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pylab as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


### Softmax

In [2]:
# tf graph input
X = tf.placeholder('float', [None, 784])
y = tf.placeholder('float', [None, 10])

# set weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# model
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# minimize cost
learning_rate = 0.1
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(hypothesis), reduction_indices=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# init
init = tf.global_variables_initializer()

# set training_epochs, batch_size, display_step
training_epochs = 15
batch_size = 100
display_step = 1

# launch
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict = {X:batch_xs, y:batch_ys})
            avg_cost += sess.run(cost, feed_dict={X:batch_xs, y:batch_ys}) / total_batch
        # Display logs
        if epoch % display_step == 0:
            print("Epoch: {:04d}, Cost= {:.9f}".format(epoch+1, avg_cost))
            
    # Test model
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    print("Accuracy:", accuracy.eval({X:mnist.test.images, y:mnist.test.labels}))

Epoch: 0001, Cost= 0.521184075
Epoch: 0002, Cost= 0.352593903
Epoch: 0003, Cost= 0.324264894
Epoch: 0004, Cost= 0.309338853
Epoch: 0005, Cost= 0.299760784
Epoch: 0006, Cost= 0.292863023
Epoch: 0007, Cost= 0.287666024
Epoch: 0008, Cost= 0.283491803
Epoch: 0009, Cost= 0.280248994
Epoch: 0010, Cost= 0.277030452
Epoch: 0011, Cost= 0.274587417
Epoch: 0012, Cost= 0.272341755
Epoch: 0013, Cost= 0.270360379
Epoch: 0014, Cost= 0.268654794
Epoch: 0015, Cost= 0.266825286
Accuracy: 0.9218


### Sfotmax with tensorbard

In [3]:
# Set parameters
learning_rate = 0.01
training_epochs = 15
batch_size = 100
display_step = 1

# tf Graph input
X = tf.placeholder("float", [None, 784], name = "X-input")
y = tf.placeholder("float", [None, 10], name = "y-input")

# Set weights
W = tf.Variable(tf.zeros([784, 10]), name = "weight")
b = tf.Variable(tf.zeros([10]), name = "bias")

# Add histogram
w_hist = tf.summary.histogram("weight", W)
b_hist = tf.summary.histogram("bias", b)
y_hist = tf.summary.histogram("y", y)


# Construct model
with tf.name_scope("layer2") as scope:
    hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Minimize Cost
with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(hypothesis), reduction_indices=1))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize variables
init = tf.global_variables_initializer()

# Launch
with tf.Session() as sess:
    
    # tensorboard merge
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/softmax_mnist_logs", sess.graph)
    
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Loop batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={X:batch_xs, y:batch_ys})
            avg_cost += sess.run(cost, feed_dict={X:batch_xs, y:batch_ys}) / total_batch
            
            # tensorbard
            if epoch % display_step == 0:
                summary, _ = sess.run([merged, optimizer], feed_dict={X:batch_xs, y:batch_ys})
                writer.add_summary(summary, epoch)
            
        # Display logs
        if epoch % display_step == 0:
            print("Epoch: {:04d}, Cost= {:.9f}".format(epoch+1, avg_cost))
        
    # Test model
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X:mnist.train.images, y:mnist.train.labels}))

Epoch: 0001, Cost= 0.922257241
Epoch: 0002, Cost= 0.524568467
Epoch: 0003, Cost= 0.452979745
Epoch: 0004, Cost= 0.417778888
Epoch: 0005, Cost= 0.395971195
Epoch: 0006, Cost= 0.380602818
Epoch: 0007, Cost= 0.368948672
Epoch: 0008, Cost= 0.359839258
Epoch: 0009, Cost= 0.352355014
Epoch: 0010, Cost= 0.346042725
Epoch: 0011, Cost= 0.340726275
Epoch: 0012, Cost= 0.336137661
Epoch: 0013, Cost= 0.331947731
Epoch: 0014, Cost= 0.328308926
Epoch: 0015, Cost= 0.325056534
Accuracy: 0.910291


### NN ( Neural Network ) with ReLu

In [4]:
# Set parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# tf Graph input
X = tf.placeholder('float', [None, 784])
y = tf.placeholder('float', [None, 10])

# Set weights & bias
W1 = tf.Variable(tf.random_normal([784, 256])) # tf.zeros() 로 초기값 주면 망함 accuracy = 0.11
W2 = tf.Variable(tf.random_normal([256, 256]))
W3 = tf.Variable(tf.random_normal([256, 10]))

b1 = tf.Variable(tf.random_normal([256]))
b2 = tf.Variable(tf.random_normal([256]))
b3 = tf.Variable(tf.random_normal([10]))

# Construct model
L1 = tf.nn.relu(tf.matmul(X, W1) + b1) # Hidden layers with ReLu activation
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
hypothesis = tf.matmul(L2, W3) + b3 # No need to use softmax here

# Minimize Cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hypothesis, y)) # Softmax loss
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initialize
init = tf.global_variables_initializer()

# Launch
with tf.Session() as sess:
    sess.run(init)
    
    # Loop over all epochs
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={X:batch_xs, y:batch_ys})
            avg_cost += sess.run(cost, feed_dict={X:batch_xs, y:batch_ys}) / total_batch
            
        # Display logs
        if epoch % display_step == 0:
            print("epoch: {:04d}, cost= {:.9f}".format(epoch+1, avg_cost))
            
    print("Optimization Finished !!!")
    
    # Test
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    print("Accuracy:", accuracy.eval({X:mnist.train.images, y:mnist.train.labels}))

epoch: 0001, cost= 170.300113751
epoch: 0002, cost= 39.842550096
epoch: 0003, cost= 24.403118906
epoch: 0004, cost= 16.874933719
epoch: 0005, cost= 12.071800723
epoch: 0006, cost= 8.843322552
epoch: 0007, cost= 6.461736077
epoch: 0008, cost= 4.745275340
epoch: 0009, cost= 3.399168718
epoch: 0010, cost= 2.416047792
epoch: 0011, cost= 1.665859365
epoch: 0012, cost= 1.192046224
epoch: 0013, cost= 0.837875080
epoch: 0014, cost= 0.573309850
epoch: 0015, cost= 0.452114851
Optimization Finished !!!
Accuracy: 0.989309


### NN with Xavier Initializer

In [5]:
def xavier_init(n_inputs, n_outputs, uniform=True):
    """Set the parameter initialization using the method described.
    This method is designed to keep the scale of the gradients roughly the same
    in all layers.
    Xavier Glorot and Yoshua Bengio (2010):
            Understanding the difficulty of training deep feedforward neural
            networks. International conference on artificial intelligence and
            statistics.
    Args:
      n_inputs: The number of input nodes into each output.
      n_outpus: The number of output nodes for each input.
      uniform: If true use a uniform distribution, otherwise use a normal.
    Returns:
      An initializer.
    """
    if uniform:
        # 6 was used in the paper
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        # 3 gives us approximately the smae limits as above since this repicks
        # values greater than 2 standard deviations from the maen.
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)
    

# Set parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# tf Graph input
X = tf.placeholder('float', [None, 784])
y = tf.placeholder('float', [None, 10])

# Set weights & biases
with tf.variable_scope("weights1") as scope:
    try:
        W1 = tf.get_variable('W1', shape=[784, 256], initializer=xavier_init(784, 256)) # get_variable
        W2 = tf.get_variable('W2', shape=[256, 256], initializer=xavier_init(256, 256))
        W3 = tf.get_variable('W3', shape=[256, 10], initializer=xavier_init(256, 10))
    except ValueError:
        scope.reuse_variables()
        W1 = tf.get_variable('W1', shape=[784, 256], initializer=xavier_init(784, 256)) 
        W2 = tf.get_variable('W2', shape=[256, 256], initializer=xavier_init(256, 256))
        W3 = tf.get_variable('W3', shape=[256, 10], initializer=xavier_init(256, 10))

b1 = tf.Variable(tf.random_normal([256]))
b2 = tf.Variable(tf.random_normal([256]))
b3 = tf.Variable(tf.random_normal([10]))

# Construct model
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
hypothesis = tf.matmul(L2, W3) + b3

# Minimize cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hypothesis, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initialize all variables
init = tf.global_variables_initializer()

# Launch
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Loop over all total_batch
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={X:batch_xs, y:batch_ys})
            avg_cost += sess.run(cost, feed_dict={X:batch_xs, y:batch_ys}) / total_batch
        # Display logs
        if epoch % display_step == 0:
            print("epoch: {:04d}, cost= {:.9f}".format(epoch+1, avg_cost))

    print("Optimization Finished !")
            
    # Test
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    print("Accuracy:", accuracy.eval({X:mnist.test.images, y:mnist.test.labels}))

epoch: 0001, cost= 0.291347500
epoch: 0002, cost= 0.097839827
epoch: 0003, cost= 0.061555574
epoch: 0004, cost= 0.041461504
epoch: 0005, cost= 0.029221285
epoch: 0006, cost= 0.020642745
epoch: 0007, cost= 0.014702703
epoch: 0008, cost= 0.011271844
epoch: 0009, cost= 0.008376830
epoch: 0010, cost= 0.007577952
epoch: 0011, cost= 0.005036811
epoch: 0012, cost= 0.005082444
epoch: 0013, cost= 0.005003386
epoch: 0014, cost= 0.003872608
epoch: 0015, cost= 0.002858293
Optimization Finished !
Accuracy: 0.9755


### NN with Deeper & Dropout

In [6]:
# Xavier Initializer
def xavier_init(n_inputs, n_outputs, uniform=True):
    """Set the parameter initialization using the method described.
    This method is designed to keep the scale of the gradients roughly the same
    in all layers.
    Xavier Glorot and Yoshua Bengio (2010):
            Understanding the difficulty of training deep feedforward neural
            networks. International conference on artificial intelligence and
            statistics.
    Args:
      n_inputs: The number of input nodes into each output.
      n_outpus: The number of output nodes for each input.
      uniform: If true use a uniform distribution, otherwise use a normal.
    Returns:
      An initializer.
    """
    if uniform:
        # 6 was used in the paper
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        # 3 gives us approximately the smae limits as above since this repicks
        # values greater than 2 standard deviations from the maen.
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

In [7]:
# Set parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# tf graph input
X = tf.placeholder('float', [None, 784])
y = tf.placeholder('float', [None, 10])

# Store layers weights & biases
with tf.variable_scope("weights2") as scope:
    try:
        W1 = tf.get_variable('W1', shape=[784, 256], initializer=xavier_init(784, 256))
        W2 = tf.get_variable('W2', shape=[256, 256], initializer=xavier_init(256, 256))
        W3 = tf.get_variable('W3', shape=[256, 256], initializer=xavier_init(256, 256))
        W4 = tf.get_variable('W4', shape=[256, 128], initializer=xavier_init(256, 128))
        W5 = tf.get_variable('W5', shape=[128, 10], initializer=xavier_init(128, 10))
    except ValueError:
        scope.reuse_variables()
        W1 = tf.get_variable('W1', shape=[784, 256], initializer=xavier_init(784, 256))
        W2 = tf.get_variable('W2', shape=[256, 256], initializer=xavier_init(256, 256))
        W3 = tf.get_variable('W3', shape=[256, 256], initializer=xavier_init(256, 256))
        W4 = tf.get_variable('W4', shape=[256, 128], initializer=xavier_init(256, 128))
        W5 = tf.get_variable('W5', shape=[128, 10], initializer=xavier_init(128, 10))
        
b1 = tf.Variable(tf.random_normal([256]))
b2 = tf.Variable(tf.random_normal([256]))
b3 = tf.Variable(tf.random_normal([256]))
b4 = tf.Variable(tf.random_normal([128]))
b5 = tf.Variable(tf.random_normal([10]))

# Construct model with dropout
dropout_rate = tf.placeholder('float')

_L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(_L1, dropout_rate)
_L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(_L2, dropout_rate)
_L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(_L3, dropout_rate)
_L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(_L4, dropout_rate)

hypothesis = tf.matmul(L4, W5) + b5

# Minimize cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(hypothesis, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initialize
init = tf.global_variables_initializer()

# Launch
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Trai, Loop over tatal batch
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={X:batch_xs, y:batch_ys, dropout_rate:0.7})
            avg_cost += sess.run(cost, feed_dict={X:batch_xs, y:batch_ys, dropout_rate:0.7}) / total_batch
        # Display logs    
        if epoch % display_step == 0:
            print("epoch: {:04d}, cost={:.9f}".format(epoch+1, avg_cost))
            
    print("Optimization Finished !")
    
    # Test
    correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    print("Accuaracy:", accuracy.eval({X:mnist.test.images, y:mnist.test.labels, dropout_rate:1}))

epoch: 0001, cost=0.614708607
epoch: 0002, cost=0.218250300
epoch: 0003, cost=0.162101555
epoch: 0004, cost=0.136621421
epoch: 0005, cost=0.117607634
epoch: 0006, cost=0.106442019
epoch: 0007, cost=0.095276398
epoch: 0008, cost=0.088351584
epoch: 0009, cost=0.079378287
epoch: 0010, cost=0.074049040
epoch: 0011, cost=0.070707715
epoch: 0012, cost=0.065686001
epoch: 0013, cost=0.065092079
epoch: 0014, cost=0.058808897
epoch: 0015, cost=0.055060618
Optimization Finished !
Accuaracy: 0.9809
